In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
from keras.models import load_model
import json
import random
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import layers
import speech_recognition as sr
import pyttsx3
import os
import datetime
import wikipedia
import webbrowser
import smtplib
import json 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from selenium import webdriver

In [2]:
lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_words = ['?','!']
with open("intents.json") as file:
    data = json.load(file)
    

for intent in data['intents']:
    
    for pattern in intent['patterns']:
        
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        for i in w:
            words.append(i)
        
        #add documents in the corpus
        documents.append((w, intent['tag']))
        
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

#lemmatize each word
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

#sort class
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

33 documents
8 classes ['about', 'complaint', 'createaccount', 'goodbye', 'greeting', 'help', 'name', 'thanks']
52 unique lemmatized words ["'s", 'a', 'about', 'account', 'an', 'anyone', 'are', 'bye', 'call', 'can', 'complaint', 'could', 'create', 'do', 'for', 'give', 'goodbye', 'hand', 'have', 'hay', 'hello', 'help', 'helpful', 'hey', 'hi', 'how', 'i', 'is', 'later', 'me', 'name', 'need', 'new', 'open', 'please', 'raise', 'see', 'service', 'should', 'support', 'thank', 'thanks', 'that', 'the', 'there', 'to', 'want', 'what', 'whats', 'who', 'you', 'your']


In [3]:
#create training data
training = []

# create an empty array for output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    
    # initialize our bag of words
    bag = []
    
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
    
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\temp\ipykernel_16672\3187314152.py:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [4]:
# Create a model of 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
print("model created")

Epoch 1/200
7/7 [==============================] - 1s 5ms/step - loss: 2.1479 - accuracy: 0.0606
Epoch 2/200
7/7 [==============================] - 0s 5ms/step - loss: 2.1319 - accuracy: 0.0606
Epoch 3/200
7/7 [==============================] - 0s 5ms/step - loss: 1.9914 - accuracy: 0.2121
Epoch 4/200
7/7 [==============================] - 0s 5ms/step - loss: 1.9102 - accuracy: 0.3030
Epoch 5/200
7/7 [==============================] - 0s 5ms/step - loss: 1.8604 - accuracy: 0.3636
Epoch 6/200
7/7 [==============================] - 0s 3ms/step - loss: 1.7745 - accuracy: 0.3030
Epoch 7/200
7/7 [==============================] - 0s 4ms/step - loss: 1.6675 - accuracy: 0.4242
Epoch 8/200
7/7 [==============================] - 0s 6ms/step - loss: 1.7228 - accuracy: 0.3333
Epoch 9/200
7/7 [==============================] - 0s 5ms/step - loss: 1.4503 - accuracy: 0.5152
Epoch 10/200
7/7 [==============================] - 0s 4ms/step - loss: 1.4129 - accuracy: 0.5152
Epoch 11/200
7/7 [===========

7/7 [==============================] - 0s 3ms/step - loss: 0.0195 - accuracy: 1.0000
Epoch 85/200
7/7 [==============================] - 0s 3ms/step - loss: 0.0696 - accuracy: 1.0000
Epoch 86/200
7/7 [==============================] - 0s 3ms/step - loss: 0.1075 - accuracy: 0.9697
Epoch 87/200
7/7 [==============================] - 0s 3ms/step - loss: 0.0835 - accuracy: 1.0000
Epoch 88/200
7/7 [==============================] - 0s 3ms/step - loss: 0.1261 - accuracy: 0.9394
Epoch 89/200
7/7 [==============================] - 0s 3ms/step - loss: 0.0596 - accuracy: 1.0000
Epoch 90/200
7/7 [==============================] - 0s 3ms/step - loss: 0.1751 - accuracy: 0.9394
Epoch 91/200
7/7 [==============================] - 0s 5ms/step - loss: 0.0283 - accuracy: 1.0000
Epoch 92/200
7/7 [==============================] - 0s 5ms/step - loss: 0.0358 - accuracy: 1.0000
Epoch 93/200
7/7 [==============================] - 0s 4ms/step - loss: 0.0764 - accuracy: 1.0000
Epoch 94/200
7/7 [===============

7/7 [==============================] - 0s 4ms/step - loss: 0.0104 - accuracy: 1.0000
Epoch 168/200
7/7 [==============================] - 0s 5ms/step - loss: 0.0733 - accuracy: 0.9697
Epoch 169/200
7/7 [==============================] - 0s 6ms/step - loss: 0.0065 - accuracy: 1.0000
Epoch 170/200
7/7 [==============================] - 0s 4ms/step - loss: 0.0086 - accuracy: 1.0000
Epoch 171/200
7/7 [==============================] - 0s 5ms/step - loss: 0.0085 - accuracy: 1.0000
Epoch 172/200
7/7 [==============================] - 0s 3ms/step - loss: 0.0115 - accuracy: 1.0000
Epoch 173/200
7/7 [==============================] - 0s 5ms/step - loss: 0.0079 - accuracy: 1.0000
Epoch 174/200
7/7 [==============================] - 0s 5ms/step - loss: 0.0138 - accuracy: 1.0000
Epoch 175/200
7/7 [==============================] - 0s 3ms/step - loss: 0.0066 - accuracy: 1.0000
Epoch 176/200
7/7 [==============================] - 0s 3ms/step - loss: 0.0298 - accuracy: 1.0000
Epoch 177/200
7/7 [=====

In [5]:
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [6]:
class chatBot():
    def __init__(self, name):
        print(f'Starting up, {name}')
        self.name = name        
        
    def takeCommand(self):
        r = sr.Recognizer()
        with sr.Microphone() as source:
            r.adjust_for_ambient_noise(source)
            print('Listening...')
            r.pause_threshold = 1
            audio = r.listen(source)
        try:
            print('Recognizing..')
            query = r.recognize_google(audio, language = 'en-in')
            print(f'User said: {query}\n')
        except Exception as e:
            print(e)
            print('Could you repeat that again please...')
            return 'none'
        return query
        
    def wake_up(self, text):
        if self.name.lower() in text:
            return True
        else:
            return False
    
    def speak(self, text):
        engine = pyttsx3.init('sapi5')
        voices = engine.getProperty('voices')
        engine.setProperty('voice', voices[2].id)
        engine.say(text)
        print("AI --> ", text)
        engine.runAndWait()
        
    def wishMe(self):
        hour = int(datetime.datetime.now().hour)
        
        if hour >= 0 and hour < 12:
            ai.speak('Good Morning, Master Wayne!')

        elif hour >= 12 and hour < 18:
            ai.speak('Good Afternoon, Master Wayne!')

        else:
            ai.speak('Good Evening, Master Wayne!')
            
    def filteredQuery(self, query):
        stop_words = set(stopwords.words('english'))
        word_tokens = word_tokenize(query)
        filtered_query = [w for w in word_tokens if not w.lower() in stop_words]
        bannedWord = ['google','search','open', 'start', 'youtube', 'play', 'wikipedia']
        query_1 = str()
        query_1 = (' '.join(w for w in filtered_query if w not in bannedWord))
        return query_1
            
    def action_time(self):
        return datetime.datetime.now().time().strftime('%H:%M')
            
    def clean_up_sentence(self, sentence):
        # tokenize the pattern - split words into array
        sentence_words = nltk.word_tokenize(sentence)
        # stem each word - create short form for word
        sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
        return sentence_words
        # return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

    def bow(self, sentence, words, show_details=True):
        # tokenize the pattern
        sentence_words = ai.clean_up_sentence(sentence)
        # bag of words - matrix of N words, vocabulary matrix
        bag = [0]*len(words) 
        for s in sentence_words:
            for i,w in enumerate(words):
                if w == s: 
                    # assign 1 if current word is in the vocabulary position
                    bag[i] = 1
                    if show_details:
                        print ("found in bag: %s" % w)
        return(np.array(bag))

    def predict_class(self, sentence, model):
        # filter out predictions below a threshold
        p = ai.bow(sentence, words,show_details=False)
        res = model.predict(np.array([p]))[0]
        ERROR_THRESHOLD = 0.25
        results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
        # sort by strength of probability
        results.sort(key=lambda x: x[1], reverse=True)
        return_list = []
        for r in results:
            #r[1] = str(r[1])
            return_list.append({"intent": classes[r[0]], "probability": r[1]})
        return return_list
    
    def getResponse(self, ints, intents_json):
        tag = ints[0]['intent']
        list_of_intents = intents_json['intents']
        for i in list_of_intents:
            if(i['tag']== tag):
                result = random.choice(i['responses'])
                break
        return result

    def chatbot_response(self, text):
        ints = ai.predict_class(text, model)
        res = ai.getResponse(ints, intents)
        return res

In [8]:
if __name__ == '__main__':
    ai = chatBot(name = 'Joanna')
    while True:
        text = ai.takeCommand().lower()
        res = ''
        
        if ai.wake_up(text) == True:
            ai.wishMe()
            res = "I am Joanna the AI. what can I do for you?"
        
        elif 'time' in text:
            res = ai.action_time()
        
        elif 'wikipedia' in text:
            ai.speak('Searching wikipedia...')
            text = text.replace('wikipedia', '')
            results = wikipedia.summary(text, sentences = 2)
            ai.speak('According to wikipedia')
            ai.speak(results)
            print(f'According to wikipedia, {results}')
        
        elif 'google' in text:
            ai.speak('Opening google...')
            text = ai.filteredQuery(text)
            webbrowser.open(f'https://www.google.com/search?q={text}')
        
        elif 'youtube' in text:
            ai.speak('Opening youtube...')
            text = ai.filteredQuery(text)
            webbrowser.open('youtube.com')
        
        elif 'music' in text:
            ai.speak('Opening Amazon prime music...')
            webbrowser.open('https://music.amazon.in/stations/A1ESXGJW9GSMCX?marketplaceId=A3K6Y4MI8GDYMT&musicTerritory=IN&ref=dm_sh_b0bokLtyOS6TlPaJjJxe0uCVE')
        
        elif 'stack overflow' in text:
            ai.speak('Opening stack overflow...')
            webbrowser.open('stackoverflow.com')
                
        else:
            res = ai.chatbot_response(text)
        ai.speak(res)
        

Starting up, Joanna
Listening...
Recognizing..
User said: I join

AI -->  Hello
Listening...
Recognizing..
User said: bhai joiner

AI -->  Hi
Listening...
Recognizing..
User said: hi Jana

AI -->  Hi
Listening...
Recognizing..
User said: what's the time

AI -->  10:00
Listening...
Recognizing..
User said: what can you do for me

AI -->  Tell me your problem to assist you
Listening...
Recognizing..
User said: hi Joanna

AI -->  Good Morning, Master Wayne!
AI -->  I am Joanna the AI. what can I do for you?
Listening...


KeyboardInterrupt: 

In [ ]:
#self.driver = webdriver.Firefox(executable_path=r"C:\Users\pratham soneja\OneDrive\Documents\geckodriver.exe")
#        elif 'team' in text:
#            teamsPath = r'"C:\Users\pratham soneja\AppData\Local\Microsoft\Teams\Update.exe"'
#            os.startfile(teamsPath)
            
#        elif 'play' in query:
#            ai.speak('Opening youtube...')
#            text = ai.filteredQuery(text)
#            play(text)
            
#        elif 'send email' in text:
#            try:
#                ai.speak('What should i say?')
#                content = ai.takeCommand()
#                to = 'pratham.soneja@yahoo.com'
#                ai.sendEmail(to, content)
#                ai.speak('Email has been sent!')
#            except Exceptioin as e:
#                print(e)
#                ai.speak('Sorry, could not send the mail') 

#    def sendEmail(self, to, content):
#        server = smtplib.SMTP('smtp.gmail.com', 587)
#        server.ehlo()
#        server.starttls()
#        server.login('pratham.soneja.pcm@gmail.com', 'your-password')
#        server.sendmail('pratham.soneja.pcm@gmail.com', to, content)
#        server.quit()
        
#    def play(self,query):
#        self.query=query
#        self.driver.get(url="https://www.youtube.com/results?search_query=" + query)
#        self.driver.find_element_by_xpath('//*[@id="video-title"/yt-formatted-string')
